In [ ]:
from GA.ga_main import ga_main
import numpy as np
import os
import matplotlib.pyplot as plt

## Example of permutation coding.
### Solve traveling salesman problem

In [ ]:
def plot_root(gen,count):
    plt.scatter(0,0,c="g")
    plt.scatter(position[:,0],position[:,1],c="b")
    plt.plot([0,position[ga.best_ind[0],0]],[0,position[ga.best_ind[0],1]],c = "r")
    plt.plot([0,position[ga.best_ind[-1],0]],[0,position[ga.best_ind[-1],1]],c = "r")
    plt.plot(position[ga.best_ind,0],position[ga.best_ind,1],c ="r")
    plt.title("Generation :{}".format(gen))
    plt.savefig("./tsp_graph/root_plot_{0:04d}.png".format(count),format="png")
    plt.close()

In [ ]:
"""initialize"""
n_parents = 15        # the number of parents
l_gene = 50           # the length of gen
n_pop   = 100         # the number of popuration
e_size = 5            # Elete size for elete selection
t_size =10            # tournament size for tournament selection

pb_crs = 1            # probability of crossover
pb_mut = 0.1          # probability of mutation

ga = ga_main(l_gene = l_gene, n_pop = n_pop, n_parents=n_parents,  # Initialize
             e_size=e_size, pb_mut=pb_mut, pb_crs = pb_crs) 

"""Ratios of each methods in operations"""
crs_ratio = [0,0,1,0]  # [cycle, op_order, order_based, position_based]
mut_ratio = [0.25,0.25,0.25,0.25] # [swap, inversion, scramble, translocation]

"""Set parameters"""
ga.set_params(crs_ratio = crs_ratio, mut_ratio = mut_ratio)

ga.show_params()  # print all parameters

max_generation = 1500 # Max generation

In [ ]:
"""Initial position"""
position = np.random.rand(l_gene,2)*100 

stop_limit = 100
count = 0
stop_count = 0

for gen in range(1,max_generation):
    print(" ----------  Generation : {} ---------".format(gen))
    if gen == 1:
        ga.make_init_generation(data_type="permutation") # make 1st generation population
        fitness = ga.calc_dist_fitness(position)
        ga.get_fitness(fitness)                          # calculate fitness
        print("best fitness : "+ str(max(ga.fitness)))
        ga.get_best_individuals()                        # get best individual in this generation
        count = count +1
        plot_root(gen,count)
        continue
    
    
    """Selection"""
    ### Tournament selection
    parents = ga.tournament_selection(t_size=t_size, p_size=ga.n_parents, 
                                       fitness=ga.fitness, population = ga.inds)
    ### Elete selection
    kb = ga.elete_selection(e_size=e_size, fitness=ga.fitness, population = ga.inds)   
    
     
    offspring = np.empty([0,ga.l_gene]) ### next generation population
    while offspring.shape[0] != ga.n_pop-ga.e_size:
        p1, p2 = np.random.choice(parents.shape[0],2,replace=False) 
        """Crossover"""
        ga.Crossover(parents[p1], parents[p2])
        """Mutation"""
        ga.Mutation(parent=ga.child)
        #ga.GMutation(parent=ga.child, generation=gen, extra_mut=0.5, cycle=100)
        offspring = np.append(offspring, ga.child.reshape(1,-1),axis=0)
    
    
    offspring = np.append(offspring,kb,axis=0)
    ga.inds = offspring.astype(int)
    
    fitness = ga.calc_dist_fitness(position)
    ga.get_fitness(fitness)                          # calculate fitness
    print("best fitness : "+ str(max(ga.fitness)))
    ga.get_best_individuals()
    
    """Plot graph"""
    if ga.best_fit_list[gen-2] < ga.best_fit_list[gen-1]:
        count = count +1
        plot_root(gen,count)
    
    """Check converge"""
    if ga.best_fit_list[gen-1] == ga.best_fit_list[gen-2]:
        stop_count = stop_count +1
    elif ga.best_fit_list[gen-1] != ga.best_fit_list[gen-2]:
        stop_count = 0
    
    if stop_count == stop_limit:
        print("GA is converged. Finish evolutional roop.")
        break
        
    

In [ ]:
%matplotlib inline

plt.scatter(0,0,c="g")
plt.scatter(position[:,0],position[:,1],c="b")
plt.plot([0,position[ga.best_ind[0],0]],[0,position[ga.best_ind[0],1]],c = "r")
plt.plot([0,position[ga.best_ind[-1],0]],[0,position[ga.best_ind[-1],1]],c = "r")
plt.plot(position[ga.best_ind,0],position[ga.best_ind,1],c ="r")


In [ ]:
"""Make gif"""
import os
os.system("convert -layers optimize -loop 0 -delay 40 ./tsp_graph/root_plot_*.png ./tsp_graph/root_plot.gif")

## Example of binary coding.
### Solve OneMax problem

In [ ]:
"""initialize"""
n_parents = 15        # the number of parents
l_gene = 200            # the length of gen
n_pop   = 100         # the number of popuration
e_size = 5            # Elete size for elete selection
t_size =10            # tournament size for tournament selection

pb_crs = 1            # probability of crossover
pb_mut = 0.1          # probability of mutation

ga = ga_main(l_gene = l_gene, n_pop = n_pop, n_parents=n_parents,  
             e_size=e_size, pb_mut=pb_mut, pb_crs = pb_crs, func_set= "binary") 

"""Ratios of each methods in operations"""
crs_ratio = [0,1,0]  # [op, tp, uniform]
mut_ratio = [0.25,0.25,0.25,0.25] # [substitution, inversion, scramble, translocation]

"""Set parameters"""
ga.set_params(crs_ratio = crs_ratio, mut_ratio = mut_ratio)

ga.show_params()  # print all parameters

max_generation = 1500 # Max generation

In [ ]:
"""Initial position"""
stop_limit = 100
stop_count = 0

for gen in range(1,max_generation):
    print(" ----------  Generation : {} ---------".format(gen))
    if gen == 1:
        ga.make_init_generation(data_type="binary", n_1 = "random") # make 1st generation population
        fitness = ga.calc_onemax_fitness()
        ga.get_fitness(fitness)                          # calculate fitness
        print("best fitness : "+ str(max(ga.fitness)))
        ga.get_best_individuals()                        # get best individual in this generation
        continue
    
    
    """Selection"""
    ### Tournament selection
    parents = ga.tournament_selection(t_size=t_size, p_size=ga.n_parents, 
                                       fitness=ga.fitness, population = ga.inds)
    ### Elete selection
    kb = ga.elete_selection(e_size=e_size, fitness=ga.fitness, population = ga.inds)   
    
     
    offspring = np.empty([0,ga.l_gene]) ### next generation population
    while offspring.shape[0] != ga.n_pop-ga.e_size:
        p1, p2 = np.random.choice(parents.shape[0],2,replace=False) 
        """Crossover"""
        ga.Crossover(parents[p1], parents[p2])
        """Mutation"""
        ga.Mutation(parent=ga.child)
        #ga.GMutation(parent=ga.child, generation=gen, extra_mut=0.5, cycle=100)
        offspring = np.append(offspring, ga.child.reshape(1,-1),axis=0)
    
    
    offspring = np.append(offspring,kb,axis=0)
    ga.inds = offspring.astype(int)
    
    fitness = ga.calc_onemax_fitness()
    ga.get_fitness(fitness)                          # calculate fitness
    print("best fitness : "+ str(max(ga.fitness)))
    ga.get_best_individuals()
    
    """Check converge"""
    if ga.best_fit_list[gen-1] == ga.best_fit_list[gen-2]:
        stop_count = stop_count +1
    elif ga.best_fit_list[gen-1] != ga.best_fit_list[gen-2]:
        stop_count = 0
    
    if stop_count == stop_limit:
        print("GA is converged. Finish evolutional roop.")
        break
        
    